In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2024,1)
bins=np.arange(0,1050,50)

ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas_merra2=ds['cell_area']/1e6

ds=xr.open_dataset(dir_data+'jra55_gridareas.nc')
areas_jra55=ds['cell_area']/1e6

area_land_era5=np.zeros((len(yrs),len(bins)-1))
area_land_jra55=np.zeros((len(yrs),len(bins)-1))
area_land_merra2=np.zeros((len(yrs),len(bins)-1))
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
            
    ds=xr.open_dataset(dir3+'jra55_ivt_'+str(year)+'.nc')
    ivt_jra55=ds['ivt']
    
    ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
    ivt_merra2=ds['ivt'][0::2,:,:]
    
    ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
    ivt_era5=ds['ivt']
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)

    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')

    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values

    land_mask=dk['land']
    land_mask_era5=land_mask.interp_like(ivt_era5[0,:,:])
    land_mask_merra2=land_mask.interp_like(ivt_merra2[0,:,:])
    land_mask_jra55=land_mask.interp_like(ivt_jra55[0,:,:])
    
    lats=ivt_era5.lat.values
    lons=ivt_era5.lon.values

    ivt_era5=ivt_era5.where(land_mask_era5<0.5)
    ivt_era5=ivt_jra55.where(land_mask_jra55<0.5)
    ivt_era5=ivt_merra2.where(land_mask_merra2<0.5)

    ivt_era5['lat']=lats
    ivt_era5['lon']=lons
    ivt_merra2['lat']=lats
    ivt_merra2['lon']=lons
    areas_merra2['lat']=lats
    areas_merra2['lon']=lons
    
    lats_sel_merra2=[x for x in ivt_merra2.lat.values if x>=23.5 or x<=-23.5]
    lats_sel_era5=[x for x in ivt_era5.lat.values if x>=23.5 or x<=-23.5]
    lats_sel_jra55=[x for x in ivt_jra55.lat.values if x>=23.5 or x<=-23.5]
    
    #lats_sel_merra2=[x for x in ivt_merra2.lat.values if -23.5<x<23.5]
    #lats_sel_era5=[x for x in ivt_era5.lat.values if -23.5<x<23.5]
    #lats_sel_jra55=[x for x in ivt_jra55.lat.values if -23.5<x<23.5]
    
    for j in range(len(bins)-1):
        print(j)
        ivt_land=ivt_era5.where(ivt_era5>=bins[j]).where(ivt_era5<bins[j+1])
        ivt_land=ivt_land/ivt_land
        ivt_land=ivt_land.sum(dim='time',skipna=True)
        area_land=areas_merra2*ivt_land
        area_land=area_land.sel(lat=lats_sel_era5)
        area_land_era5[i,j]=area_land.sum().values.tolist()

        ivt_land=ivt_jra55.where(ivt_jra55>=bins[j]).where(ivt_jra55<bins[j+1])
        ivt_land=ivt_land/ivt_land
        ivt_land=ivt_land.sum(dim='time',skipna=True)
        area_land=areas_jra55*ivt_land
        area_land=area_land.sel(lat=lats_sel_jra55)
        area_land_jra55[i,j]=area_land.sum().values.tolist()
   
        ivt_land=ivt_merra2.where(ivt_merra2>=bins[j]).where(ivt_merra2<bins[j+1])
        ivt_land=ivt_land/ivt_land
        ivt_land=ivt_land.sum(dim='time',skipna=True)
        area_land=areas_merra2*ivt_land
        area_land=area_land.sel(lat=lats_sel_merra2)
        area_land_merra2[i,j]=area_land.sum().values.tolist()

In [ ]:
dk=xr.Dataset()
dk['area_land_merra2']=(('time','bins'),area_land_merra2)
dk['area_land_era5']=(('time','bins'),area_land_era5)
dk['area_land_jra55']=(('time','bins'),area_land_jra55)
dk.coords['time']=yrs
dk.coords['bins']=np.arange(0,1000,50)
dk.to_netcdf(dir_data+'ivt_land_series_extratropical_ocean.nc',mode='w',format='NETCDF4')

In [ ]:
#ds=xr.open_dataset(dir_data+'ivt_land_series_extratropical_ocean.nc')
ds=xr.open_dataset(dir_data+'ivt_land_series_extratropical.nc')
area_land_merra2=ds['area_land_merra2'].values
area_land_era5=ds['area_land_era5'].values
area_land_jra55=ds['area_land_jra55'].values

In [ ]:
fig=plt.figure(figsize=(12,10))
ax=plt.subplot(1,1,1)

from matplotlib.colors import ListedColormap
colors=plt.cm.RdYlBu_r(np.linspace(0,1,20))

width=4
alpha=0.5
for i in range(7,len(bins)-1):
    test=area_land_merra2[:,i]
    mean=float(sum(test))/float(len(test))
    normalized=[x/mean for x in test]
    ax.plot(yrs,normalized,color=colors[i],linewidth=width,alpha=alpha)
    
    test=area_land_jra55[:,i]
    mean=float(sum(test))/float(len(test))
    normalized=[x/mean for x in test]
    ax.plot(yrs,normalized,color=colors[i],linewidth=width,alpha=alpha)
    
    test=area_land_era5[:,i]
    mean=float(sum(test))/float(len(test))
    normalized=[x/mean for x in test]
    ax.plot(yrs,normalized,color=colors[i],linewidth=width,alpha=alpha)
       
for i in range(7):
    test=area_land_merra2[:,i]
    mean=float(sum(test))/float(len(test))
    normalized=[x/mean for x in test]
    ax.plot(yrs,normalized,color=colors[i],linewidth=width,alpha=alpha)
    
    test=area_land_jra55[:,i]
    mean=float(sum(test))/float(len(test))
    normalized=[x/mean for x in test]
    ax.plot(yrs,normalized,color=colors[i],linewidth=width,alpha=alpha)
    
    test=area_land_era5[:,i]
    mean=float(sum(test))/float(len(test))
    normalized=[x/mean for x in test]
    ax.plot(yrs,normalized,color=colors[i],linewidth=width,alpha=alpha)
      
#indices=[0,3,6,9,12,15,18]  
indices=np.arange(20)
for i in range(len(indices)):
    index=indices[i]
    test1=area_land_era5[:,index]
    mean1=float(sum(test1))/float(len(test1))
    normalized1=[x/mean1 for x in test1]
    test2=area_land_jra55[:,index]  
    mean2=float(sum(test2))/float(len(test2))
    normalized2=[x/mean2 for x in test2]
    test3=area_land_merra2[:,index]  
    mean3=float(sum(test3))/float(len(test3))
    normalized3=[x/mean3 for x in test3]
    new=[(x+y+z)/3. for x,y,z in zip(normalized1,normalized2,normalized3)]
    r=st.linregress(yrs,new)
    ax.plot(yrs,[r[0]*x+r[1] for x in yrs],linewidth=2,color=colors[index])
        
ax.set_xlim(1979.5,2023.5)
ax.tick_params(labelsize=23)
ax.set_xlabel('Year',fontsize=25)
ax.set_ylabel('Change relative to mean',fontsize=25)
#ax.set_title(r'$\bf{Global}$'+' '+r'$\bf{land}$'+' '+r'$\bf{IVT}$'+' '+r'$\bf{area}$',fontsize=27,pad=5)
#ax.set_title(r'$\bf{Extratropical}$'+' '+r'$\bf{land}$'+' '+r'$\bf{IVT}$'+' '+r'$\bf{area}$',fontsize=27,pad=5)   
#ax.set_title(r'$\bf{Tropical}$'+' '+r'$\bf{land}$',fontsize=34,pad=5)   
ax.set_title(r'$\bf{Extratropical}$'+' '+r'$\bf{land}$',fontsize=36,pad=5)   

for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
ax.grid(True)
ax.set_ylim(0.401,1.599)

plt.show()

In [ ]:
fig.savefig(dir_data+'ivt_overview_4.png')

In [ ]:
sample=np.ones((100,100))*-1
fig=plt.figure(figsize=(12,10))
ax=plt.subplot(1,1,1)
cax=ax.contourf(np.arange(100),np.arange(100),sample,levels=np.arange(0,1050,50),cmap=plt.cm.RdYlBu_r)
cbar=plt.colorbar(cax,pad=0)
cbar.ax.tick_params(labelsize=23)
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=25)
plt.show()

In [ ]:
fig.savefig(dir_data+'ivt_overview_bar.png')